In [ ]:
import sys; sys.path.append("../automl/")

In [ ]:
from pathlib import Path
import yaml
import joblib

import pandas as pd

from src.automl.model.lama import TabularLamaNN
from src.automl.loggers import configure_root_logger
from src.automl.constants import create_ml_data_dir
from src.automl.model.metrics import RocAuc

In [4]:
create_ml_data_dir()
configure_root_logger()

# Tune and fit [LightAutoML FTTransformer](https://github.com/sb-ai-lab/LightAutoML) 
Fit LightAutoML FTTransformer on the 5-fold stratified cross-validation. Out of fold predictions are saved for further stacking/blending. 
[TabularLamaNN](https://github.com/dertty/automl/blob/hack/src/automl/model/lama/nn_lama.py) implementation from [automl](https://github.com/dertty/automl/tree/hack) is used.

**Unfortunately**, in LightAutoML training and tuning is performed simultaneously, hence it is impossible to save best LightAutoML parameters and then initialize model with these parameters for inference. The solution is to save model file (*joblib* format) and then use this model for inference. If necessary, we can provide this file together with the oof predictions.

**Note:** GPU is required to fit tabular NNs.

## Constants

In [5]:
DATA_PATH = Path("../data/")
RANDOM_SEED = 77
N_JOBS = 16
CONFIG_FILE = Path("../configs/config.yaml")

with CONFIG_FILE.open("r") as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

## Data

In [6]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc_2.parquet")
cat_columns = df_train.drop(columns=["target", "id"]).select_dtypes(int).columns.values.tolist()
X_train, y_train = df_train[cfg["selected_features"] + cat_columns], df_train["target"]

## Model

In [7]:
metric = RocAuc()

### Fit + Tune

**Important**: It is nearly impossible to fully reproduce LightAutoML trianing, because it strongly depends on the harware, resources utilization and timeout. To reproduce the results we can provide the saved file of a fitted model.

In [15]:
# model = TabularLamaNN(n_jobs=N_JOBS, task="classification", nn_name="fttransformer")
# model.tune(X_train, y_train, metric, timeout=60 * 60 * 2, categorical_features=cat_columns)
# model.verbose = 4
# oof = model.fit(X_train, y_train, categorical_features=cat_columns)

# print(metric(y_train, oof))

[2024-11-07 12:09:14,787] - [   START    ] - Fitting TabularLamaNN_fttransformer
[12:09:14] Stdout logging level is DEBUG.
[12:09:14] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[12:09:14] Task: binary

[12:09:14] Start automl preset with listed constraints:
[12:09:14] - time: 14400.00 seconds
[12:09:14] - CPU: 16 cores
[12:09:14] - memory: 16 GB

[12:09:14] Train data shape: (413194, 63)

[12:09:30] Feats was rejected during automatic roles guess: []
[12:09:30] Layer 1 train process start. Time left 14384.16 secs
[12:09:40] number of text features: 0 
[12:09:40] number of categorical features: 4 
[12:09:40] number of continuous features: 58 
[12:09:40] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_fttransformer_0 ...
[12:09:40] Training params: {'num_workers': 0, 'pin_memory': False, 'max_length': 256, 'is_snap': False, 'input_bn': False, 'max_emb_size': 256, 'bert_name': None, 'pooling': 'cls', 'device': device(type='cuda', index=0), 

[12:36:38] Early stopping: val loss: 0.18867024779319763, val metric: 0.8045885141339955
[12:36:39] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_TorchNN_fttransformer_0 =====
[12:37:11] Epoch: 0, train loss: 0.2057550996541977, val loss: 0.19623930752277374, val metric: 0.782837355805784
[12:37:43] Epoch: 1, train loss: 0.19530603289604187, val loss: 0.19323384761810303, val metric: 0.7896652615900496
[12:38:16] Epoch: 2, train loss: 0.19324208796024323, val loss: 0.19249777495861053, val metric: 0.7934528948144519
[12:38:48] Epoch: 3, train loss: 0.19227632880210876, val loss: 0.1917622834444046, val metric: 0.7957923109148619
[12:39:20] Epoch: 4, train loss: 0.19157063961029053, val loss: 0.1911332756280899, val metric: 0.7960408848759972
[12:39:52] Epoch: 5, train loss: 0.19133360683918, val loss: 0.19054950773715973, val metric: 0.7977637171557741
[12:40:24] Epoch: 6, train loss: 0.19086650013923645, val loss: 0.19011658430099487, val metric: 0.7987780646674585
[12:40:57]

[13:13:51] Epoch: 18, train loss: 0.18879841268062592, val loss: 0.189937561750412, val metric: 0.798740517726267
[13:14:24] Epoch: 19, train loss: 0.18872402608394623, val loss: 0.19002273678779602, val metric: 0.7985878091654215
[13:14:56] Epoch: 20, train loss: 0.1886286586523056, val loss: 0.19030529260635376, val metric: 0.7985609584682656
[13:15:28] Epoch: 21, train loss: 0.18857918679714203, val loss: 0.190310999751091, val metric: 0.798711643272844
[13:16:00] Epoch: 22, train loss: 0.18846723437309265, val loss: 0.19084405899047852, val metric: 0.7981742825884232
[13:16:33] Epoch: 23, train loss: 0.18835017085075378, val loss: 0.19038909673690796, val metric: 0.7981789339318174
[13:17:05] Epoch: 24, train loss: 0.1882804036140442, val loss: 0.19069109857082367, val metric: 0.79844290953511
[13:17:37] Epoch: 25, train loss: 0.18800267577171326, val loss: 0.18971383571624756, val metric: 0.7998962368084777
[13:18:09] Epoch: 26, train loss: 0.18795877695083618, val loss: 0.1898001

[13:47:55] Epoch: 37, train loss: 0.1876065582036972, val loss: 0.18858636915683746, val metric: 0.8051410960927423
[13:48:17] Epoch: 38, train loss: 0.18764671683311462, val loss: 0.18856172263622284, val metric: 0.8052514717737294
[13:48:39] Epoch: 39, train loss: 0.18763351440429688, val loss: 0.1883734166622162, val metric: 0.8062539283974905
[13:49:02] Epoch: 40, train loss: 0.18761155009269714, val loss: 0.18858130276203156, val metric: 0.805409352430737
[13:49:24] Epoch: 41, train loss: 0.18743589520454407, val loss: 0.18902719020843506, val metric: 0.8058298147699008
[13:49:46] Epoch: 42, train loss: 0.18748611211776733, val loss: 0.18858389556407928, val metric: 0.8055390272141942
[13:50:08] Epoch: 43, train loss: 0.18732917308807373, val loss: 0.18835480511188507, val metric: 0.8058904406574705
[13:50:30] Epoch: 44, train loss: 0.18742917478084564, val loss: 0.1883849799633026, val metric: 0.8065035394377298
[13:50:53] Epoch: 45, train loss: 0.187202587723732, val loss: 0.188

### Alternatively, load the fitted model

**GPU is required.**

In [ ]:
model = joblib.load(DATA_PATH / "models" / "lamann_fttransformer_8050_full_dataset" / "lamann_fttransformer_8050_full_dataset.joblib")

### Save model file, parameters, test and oof predictions.

In [ ]:
MODEL_NAME = "lamann_fttransformer_8050_full_dataset"
MODEL_DIR = DATA_PATH / "models" / MODEL_NAME
MODEL_DIR.mkdir(exist_ok=True)

In [ ]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)